In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)

# Normalize images
x_train, x_test = x_train / 255.0, x_test / 255.0

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False  # Freeze base model

# Modify the model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=outputs)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
datagen.fit(x_train)

# Train model
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                    validation_data=(x_test, y_test),
                    epochs=10)

# Fine-tune: Unfreeze some layers
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train again with fine-tuning
history_fine = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                         validation_data=(x_test, y_test),
                         epochs=10)

# Save model
model.save("fine_tuned_cnn.h5")

# Load and test the model
from tensorflow.keras.models import load_model
model = load_model("fine_tuned_cnn.h5")
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


<ipython-input-5-5d04158fc2b9>:19: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 42ms/step - accuracy: 0.2300 - loss: 2.0972 - val_accuracy: 0.3055 - val_loss: 1.9241
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 35ms/step - accuracy: 0.2975 - loss: 1.9256 - val_accuracy: 0.3202 - val_loss: 1.8928
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 37ms/step - accuracy: 0.3102 - loss: 1.9013 - val_accuracy: 0.3265 - val_loss: 1.8714
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.3125 - loss: 1.8898 - val_accuracy: 0.3303 - val_loss: 1.8615
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.3178 - loss: 1.8775 - val_accuracy: 0.3328 - val_loss: 1.8519
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 36ms/step - accuracy: 0.3163 - loss: 1.8783 - val_accuracy: 0.3331 - val_loss: 1.8454
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.3250 - loss: 1.8709 - val_accuracy: 0.3305 - val_loss: 1.8406
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.3253 - loss: 1.8595 - 